# I didn't finish this assignment on time. I got stuck  in the data manipulation but will send what I have worked on so far.

#The plan:
1A:
-the plan is to first tokenize our data, and clean it up as much as possible. 

-count the frequency of our words in a bag of words, and runn a Tfid to weigh down words that appear in all the texts.

-Apply LSA on the Tfid, to reduce features which we can then use to in machine learning for question 3A.




1B: Run a word to vec on the tokanized data that we have before we turn it into a bag of words.

2A. Examine our Tfid since its is a count based method. Discuss its result. Since it is a count based method it means that it doesn't look at the words context, and thus could teoreticly miss alot of the meaning.


#1 Pre-processing the data

Importing the packages needed

In [0]:
#importing standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import string
import itertools
from collections import Counter


In [48]:
# Import our tokenizers and downloading the packages
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import TweetTokenizer

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
from nltk.stem.porter import PorterStemmer

import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#Instance our word tokanizer and us it to remove handels for the tweets.
tknzr=TweetTokenizer(strip_handles=True, reduce_len=True)

In [0]:
#Loading the data
df_speech =pd.read_csv('https://transfer.sh/Zgwhy/twitter_hate_speech.csv', index_col=0)

In [51]:
#Examining the data
df_speech.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [0]:
#Tokenizing our data, and turnings into lower case. This lets us manipulate it later
df_speech['tokens'] = df_speech.apply(lambda row: tknzr.tokenize(row['tweet'].lower()),axis=1)
df_speech.head(5)

In [0]:
#In order to manipuate we are going to define a function which we can call later
#This functions filters out tokens that are in the stopwords english list from nltz
def remove_stopwords(text):
   words = [w for w in text if w not in stopwords.words('english')]
   return words

In [54]:
#Now we appy the function we just wrote.
#We want to remove stopwords because the don't add any important meaning and we want to reduce the data as much as possible
df_speech['tokens'] = df_speech.apply(lambda row: remove_stopwords(row['tokens']),axis=1)
df_speech['tokens'].head(5)

0    [!, !, !, rt, :, woman, complain, cleaning, ho...
1    [!, !, !, rt, :, boy, dats, cold, ..., tyga, d...
2    [!, !, !, rt, dawg, !, !, !, rt, :, ever, fuck...
3                 [!, !, !, rt, :, look, like, tranny]
4    [!, !, !, rt, :, shit, hear, might, true, migh...
Name: tokens, dtype: object

In [0]:
#We want to lemmatize our data
#Lemmatizing is taking our data and boilings down to the rootword
#to do this we define a function again
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
  lem_text = [lemmatizer.lemmatize(i) for i in text]
  return lem_text

In [56]:
#We then apply the function we just definned.
df_speech['tokens'] = df_speech.apply(lambda row: word_lemmatizer(row['tokens']),axis=1)
df_speech['tokens'].head(5)

0    [!, !, !, rt, :, woman, complain, cleaning, ho...
1    [!, !, !, rt, :, boy, dat, cold, ..., tyga, dw...
2    [!, !, !, rt, dawg, !, !, !, rt, :, ever, fuck...
3                 [!, !, !, rt, :, look, like, tranny]
4    [!, !, !, rt, :, shit, hear, might, true, migh...
Name: tokens, dtype: object

We are now done tokenizing and manipulating our data. The only thing I couldn't do was remove the punctuation.

# Trying other stuff
I couldn't run my TF-IDF the 'list' object has no attribute 'lower' error kept me form making progress


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_speech.tokens)
pd.DataFrame(X.A, columns=vectorizer.get_feature_names())

AttributeError: ignored

#Trying different things (Topic modeling)
I tried a bunch of other models to check what worked


In [0]:
#Bag of words
from gensim.corpora.dictionary import Dictionary


In [0]:
dictionary = Dictionary(df_speech.tokens)

In [0]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [0]:
corpus = [dictionary.doc2bow(doc) for doc in df_speech.tokens]

In [0]:
corpus[100][:10]

[(4, 1),
 (21, 1),
 (27, 1),
 (29, 1),
 (37, 2),
 (38, 1),
 (73, 1),
 (101, 1),
 (210, 1),
 (276, 1)]

In [64]:
# let's fist install this nice visualizer
!pip install pyLDAvis

In [0]:
# and import it
import pyLDAvis.gensim
%matplotlib inline
pyLDAvis.enable_notebook()

In [66]:
# Let's try to visualize
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)


/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [67]:
pyLDAvis.display(lda_display)

In [68]:
# Where does text one belong to?
lda_model[corpus][2]

[(1, 0.32747653), (5, 0.3662445), (8, 0.2031378), (9, 0.07155424)]